In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score,roc_auc_score,roc_curve,classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from skopt.callbacks import DeadlineStopper, VerboseCallback, DeltaXStopper
from pycaret.classification import *
from skopt.space import Real, Categorical, Integer
from time import time
from skopt import BayesSearchCV
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
import lightgbm as lgb
import xgboost as xgb
from IPython.display import Image  
from sklearn.externals.six import StringIO
from sklearn.tree import export_graphviz
import pydot_ng as pydot
from IPython.display import Image
import pydotplus
from xgboost import XGBClassifier

In [3]:
train = pd.read_csv('train_jqd04QH.csv')
test = pd.read_csv('test_KaymcHn.csv')

In [4]:
train.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,23798,city_149,0.689,Male,Has relevent experience,no_enrollment,Graduate,STEM,3,100-500,Pvt Ltd,1,106,0
1,29166,city_83,0.923,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,<10,Funded Startup,1,69,0
2,46,city_16,0.910,NaN,Has relevent experience,no_enrollment,Graduate,STEM,6,50-99,Public Sector,2,4,0
3,18527,city_64,0.666,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,50-99,Pvt Ltd,1,26,0
4,21751,city_100,0.887,NaN,No relevent experience,no_enrollment,Masters,STEM,8,NaN,NaN,2,88,1


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18359 entries, 0 to 18358
Data columns (total 14 columns):
enrollee_id               18359 non-null int64
city                      18359 non-null object
city_development_index    18359 non-null float64
gender                    14261 non-null object
relevent_experience       18359 non-null object
enrolled_university       18017 non-null object
education_level           17902 non-null object
major_discipline          15521 non-null object
experience                18300 non-null object
company_size              13580 non-null object
company_type              13320 non-null object
last_new_job              17992 non-null object
training_hours            18359 non-null int64
target                    18359 non-null int64
dtypes: float64(1), int64(3), object(10)
memory usage: 2.0+ MB


In [6]:
train.isnull().sum()

enrollee_id                  0
city                         0
city_development_index       0
gender                    4098
relevent_experience          0
enrolled_university        342
education_level            457
major_discipline          2838
experience                  59
company_size              4779
company_type              5039
last_new_job               367
training_hours               0
target                       0
dtype: int64

In [8]:
cols = list(train.columns)
for i in cols:
    if (i !='enrollee_id'):
        print()
        print(i)
        print(train[i].nunique())


city
123

city_development_index
93

gender
3

relevent_experience
2

enrolled_university
3

education_level
5

major_discipline
6

experience
22

company_size
8

company_type
6

last_new_job
6

training_hours
241

target
2


In [19]:
cols = list(train.columns)
for i in cols:
    if (i !='enrollee_id'):
        print()
        print(i)
        print(train[i].unique())


city
['city_149' 'city_83' 'city_16' 'city_64' 'city_100' 'city_21' 'city_114'
 'city_103' 'city_97' 'city_160' 'city_65' 'city_90' 'city_75' 'city_136'
 'city_159' 'city_67' 'city_28' 'city_10' 'city_73' 'city_76' 'city_104'
 'city_27' 'city_30' 'city_61' 'city_99' 'city_41' 'city_142' 'city_9'
 'city_116' 'city_128' 'city_74' 'city_69' 'city_1' 'city_176' 'city_40'
 'city_123' 'city_152' 'city_165' 'city_89' 'city_36' 'city_44' 'city_46'
 'city_45' 'city_134' 'city_93' 'city_180' 'city_162' 'city_84' 'city_138'
 'city_173' 'city_19' 'city_71' 'city_158' 'city_23' 'city_102' 'city_105'
 'city_91' 'city_11' 'city_13' 'city_57' 'city_20' 'city_14' 'city_37'
 'city_115' 'city_50' 'city_167' 'city_12' 'city_143' 'city_126' 'city_77'
 'city_101' 'city_150' 'city_179' 'city_107' 'city_175' 'city_146'
 'city_98' 'city_166' 'city_118' 'city_129' 'city_127' 'city_24' 'city_81'
 'city_26' 'city_139' 'city_94' 'city_70' 'city_145' 'city_157' 'city_111'
 'city_78' 'city_80' 'city_33' 'city_7' 'c

In [18]:
cols = list(train.columns)
for i in cols:
    if(train[i].isnull().sum()>=1):
        print()
        print(i)
        print('null values ',train[i].isnull().sum())
        print('% of null values in ',train[i].isnull().sum()/train['enrollee_id'].count()*100)


gender
null values  4098
% of null values in  22.321477204640775

enrolled_university
null values  342
% of null values in  1.8628465602701674

education_level
null values  457
% of null values in  2.489242333460428

major_discipline
null values  2838
% of null values in  15.458358298382265

experience
null values  59
% of null values in  0.3213682662454382

company_size
null values  4779
% of null values in  26.03082956588049

company_type
null values  5039
% of null values in  27.447028705267172

last_new_job
null values  367
% of null values in  1.9990195544419629


In [22]:
train[train['last_new_job'].isnull()]

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
30,23129,city_103,0.920,NaN,No relevent experience,no_enrollment,Graduate,STEM,8,10000+,Pvt Ltd,NaN,104,0
67,23783,city_90,0.698,NaN,No relevent experience,Full time course,Graduate,STEM,2,100-500,Pvt Ltd,NaN,145,0
112,12449,city_176,0.764,NaN,Has relevent experience,NaN,Masters,STEM,4,NaN,NaN,NaN,24,0
198,1980,city_21,0.624,NaN,Has relevent experience,Full time course,Graduate,STEM,<1,<10,Pvt Ltd,NaN,92,0
224,21814,city_100,0.887,NaN,Has relevent experience,no_enrollment,Graduate,STEM,10,5000-9999,NaN,NaN,23,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18130,18444,city_114,0.926,NaN,No relevent experience,Full time course,High School,NaN,4,NaN,NaN,NaN,4,0
18211,19128,city_61,0.913,Male,No relevent experience,Full time course,Graduate,STEM,4,NaN,NaN,NaN,140,0
18246,18493,city_99,0.915,NaN,No relevent experience,Full time course,Graduate,STEM,11,NaN,NaN,NaN,24,0
18292,28252,city_21,0.624,NaN,No relevent experience,no_enrollment,Graduate,STEM,5,NaN,Public Sector,NaN,81,0


In [23]:
train[train['experience'].isnull()]

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
149,17077,city_134,0.698,NaN,No relevent experience,no_enrollment,NaN,NaN,NaN,NaN,NaN,never,106,0
940,23806,city_114,0.926,NaN,Has relevent experience,Full time course,NaN,NaN,NaN,100-500,NaN,1,35,0
1336,4823,city_123,0.738,NaN,Has relevent experience,no_enrollment,Graduate,STEM,NaN,100-500,Pvt Ltd,NaN,9,1
1445,17123,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,NaN,NaN,NaN,NaN,32,0
2031,19353,city_73,0.754,NaN,Has relevent experience,Full time course,Graduate,STEM,NaN,NaN,NaN,1,12,1
2627,20332,city_103,0.920,NaN,Has relevent experience,Full time course,Phd,STEM,NaN,10000+,Public Sector,never,56,0
2830,10582,city_36,0.893,NaN,No relevent experience,no_enrollment,High School,NaN,NaN,NaN,NaN,never,44,0
2905,14965,city_103,0.920,NaN,Has relevent experience,no_enrollment,Masters,STEM,NaN,NaN,NaN,NaN,70,0
3103,20153,city_103,0.920,NaN,No relevent experience,Full time course,Graduate,STEM,NaN,10000+,NaN,NaN,20,1
3306,21617,city_134,0.698,NaN,No relevent experience,no_enrollment,NaN,NaN,NaN,NaN,NaN,never,45,0


In [37]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18359 entries, 0 to 18358
Data columns (total 14 columns):
enrollee_id               18359 non-null int64
city                      18359 non-null object
city_development_index    18359 non-null float64
gender                    14261 non-null object
relevent_experience       18359 non-null object
enrolled_university       18017 non-null object
education_level           17902 non-null object
major_discipline          15521 non-null object
experience                18300 non-null object
company_size              13580 non-null object
company_type              13320 non-null object
last_new_job              17992 non-null object
training_hours            18359 non-null int64
target                    18359 non-null int64
dtypes: float64(1), int64(3), object(10)
memory usage: 2.0+ MB


In [40]:
train['city'].dtype

dtype('O')

In [34]:
from sklearn.preprocessing import LabelEncoder

In [44]:
lr = LabelEncoder()
cs = [i for i in train.columns if train[i].dtype == 'object']
for c in cs:
    lr.fit(list(train[c].values))
    train[c] = lr.transform(list(train[c].values))

In [45]:
train.shape

(18359, 14)

In [29]:
imputor = KNNImputer(n_neighbors=5)

In [31]:
col = ['enrollee_id', 'city', 'city_development_index', 'gender',
       'relevent_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours', 'target']

In [46]:
knn_train = pd.DataFrame(np.round(imputor.fit_transform(train)),columns=col)

In [54]:
colu = list(knn_train.columns)
for i in colu:
    if (i!='city_development_index'):
        knn_train[i]=knn_train[i].astype(int)

In [56]:
clf1 = setup(data = knn_train, 
             target = 'target',
             numeric_imputation = 'mean',
             categorical_features = ['city', 'gender','relevent_experience','enrolled_university',
                                     'education_level','major_discipline', 'experience',
                                      'company_size', 'company_type','last_new_job',], 
             ignore_features = ['enrollee_id'],
             silent = True)

 
Setup Succesfully Completed!


,Description,Value
0,session_id,535
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(18359, 14)"
4,Missing Values,False
5,Numeric Features,2
6,Categorical Features,11
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [58]:
compare_models(sort='AUC')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa
0,Extreme Gradient Boosting,0.8683,0.6551,0.0047,0.55,0.0093,0.0075
1,Gradient Boosting Classifier,0.8671,0.6546,0.0071,0.36,0.0137,0.0085
2,Logistic Regression,0.8669,0.65,0.0029,0.25,0.0058,0.0023
3,Ada Boost Classifier,0.8666,0.6492,0.0071,0.2957,0.0137,0.0076
4,CatBoost Classifier,0.8672,0.648,0.0124,0.468,0.0239,0.0164
5,Linear Discriminant Analysis,0.861,0.6461,0.0318,0.2849,0.0568,0.0304
6,Light Gradient Boosting Machine,0.866,0.6337,0.0071,0.2337,0.0137,0.0063
7,Extra Trees Classifier,0.8532,0.6072,0.059,0.26,0.0959,0.049
8,Naive Bayes,0.2919,0.5996,0.8503,0.1404,0.241,0.0185
9,Random Forest Classifier,0.8583,0.5814,0.0413,0.2653,0.0711,0.0371


In [59]:
x = knn_train.drop(['target','enrollee_id'],axis=1)
y = knn_train['target']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=143)

In [61]:
def model_eval(algo,Xtrain,ytrain,Xtest,ytest):
    algo.fit(Xtrain,ytrain)

    y_train_pred=algo.predict(Xtrain)
    y_train_prob=algo.predict_proba(Xtrain)[:,1]
    
    y_test_pred=algo.predict(Xtest)
    y_test_prob=algo.predict_proba(Xtest)[:,1]
    print(algo)
    print('\n'*1)
    print(' Accuracy score of train: ', accuracy_score(ytrain,y_train_pred))
    print(' Accuracy score of test: ', accuracy_score(ytest,y_test_pred))
    print('\n'*1)
    print(' Confusion Matrix of train: ', confusion_matrix(ytrain,y_train_pred))
    print(' Confusion Matrix of test: ', confusion_matrix(ytest,y_test_pred))
    print('\n'*1)
    print(' Auc of train: ', roc_auc_score(ytrain,y_train_prob))
    print(' Auc of test: ', roc_auc_score(ytest,y_test_prob))

In [60]:
import xgboost as xgb
from bayes_opt import BayesianOptimization

In [65]:
xgb  = XGBClassifier()
model_eval(xgb,x_train,y_train,x_test,y_test)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)


 Accuracy score of train:  0.867948019609369
 Accuracy score of test:  0.8678286129266521


 Confusion Matrix of train:  [[11154     0]
 [ 1697     0]]
 Confusion Matrix of test:  [[4780    0]
 [ 728    0]]


 Auc of train:  0.6450331508239129
 Auc of test:  0.48723461423513725


In [66]:
import xgboost as xgb
from bayes_opt import BayesianOptimization
dtrain=xgb.DMatrix(x_train,y_train.ravel())
cv_results=[]
par=[]
def bo_tune_xgb(max_depth,min_child_weight,n_estimators):
    params = {'max_depth': int(round(max_depth)),
              'min_child_weight': int(round(min_child_weight)),
              'eval_metric':"auc",
              'n_estimators':int(round(n_estimators)),
              #"learning_rate":.1,
              #'gamma':gamma,
              'objective':'binary:logistic'
             }
    cv_result = xgb.cv(params,dtrain,num_boost_round=20,nfold=3)    
    cv_results.append(cv_result)
    par.append(params)
    #Return the negative RMSE
    print(cv_result)
    return max(cv_result["test-auc-mean"])

xgb_bo = BayesianOptimization(bo_tune_xgb, {
                                            "max_depth"        : (10,50),
                                            "min_child_weight" : (1,150),
                                            "n_estimators"     : (50,300),
                                            #"gamma"            :(0,1)
                                        })

xgb_bo.maximize(init_points=10, n_iter=30)

|   iter    |  target   | max_depth | min_ch... | n_esti... |
-------------------------------------------------------------
    train-auc-mean  train-auc-std  test-auc-mean  test-auc-std
0         0.520997       0.020715       0.498752      0.000883
1         0.580231       0.014877       0.499677      0.002993
2         0.609292       0.007478       0.497650      0.002652
3         0.620757       0.008412       0.499091      0.003234
4         0.629973       0.005190       0.498808      0.006163
5         0.633801       0.005752       0.497262      0.006705
6         0.637341       0.008421       0.500599      0.004664
7         0.639996       0.007187       0.500933      0.002651
8         0.642544       0.009742       0.501327      0.003948
9         0.647743       0.008375       0.503845      0.003998
10        0.651143       0.007181       0.503917      0.006288
11        0.654376       0.008317       0.504675      0.008248
12        0.657239       0.007399       0.502223      0.0

    train-auc-mean  train-auc-std  test-auc-mean  test-auc-std
0         0.519743       0.019014       0.498476      0.001133
1         0.550275       0.007397       0.492369      0.000800
2         0.566811       0.001648       0.496812      0.002763
3         0.572713       0.001224       0.494356      0.006697
4         0.578868       0.002524       0.495402      0.001573
5         0.581909       0.001612       0.497145      0.001173
6         0.584801       0.002673       0.500219      0.001726
7         0.588019       0.003093       0.500985      0.001688
8         0.590521       0.004068       0.499499      0.002815
9         0.590366       0.003594       0.496988      0.001460
10        0.593110       0.002710       0.497744      0.001707
11        0.595221       0.002031       0.497308      0.002956
12        0.596780       0.003422       0.498124      0.002804
13        0.598496       0.002108       0.499290      0.002310
14        0.599168       0.002009       0.500173      0

    train-auc-mean  train-auc-std  test-auc-mean  test-auc-std
0         0.547567       0.041607       0.497077      0.004778
1         0.700031       0.022897       0.494044      0.011976
2         0.808518       0.014029       0.487391      0.001487
3         0.859769       0.001676       0.483854      0.002840
4         0.880703       0.007616       0.483002      0.004921
5         0.913486       0.001240       0.487617      0.006760
6         0.927972       0.003928       0.490437      0.006434
7         0.940106       0.001858       0.494436      0.007318
8         0.951840       0.002773       0.496698      0.005215
9         0.959360       0.004759       0.497158      0.004529
10        0.966057       0.004043       0.497739      0.006326
11        0.972418       0.003957       0.501196      0.004830
12        0.974636       0.004304       0.499263      0.005346
13        0.978373       0.003398       0.498689      0.004617
14        0.981752       0.001858       0.497703      0

    train-auc-mean  train-auc-std  test-auc-mean  test-auc-std
0         0.519743       0.019014       0.498476      0.001133
1         0.548002       0.006959       0.493611      0.001581
2         0.565036       0.001534       0.496487      0.002232
3         0.568835       0.002173       0.496701      0.006005
4         0.572453       0.000948       0.498599      0.005499
5         0.575439       0.001543       0.496864      0.009514
6         0.576649       0.001167       0.498268      0.006990
7         0.577425       0.001815       0.498991      0.003886
8         0.579199       0.000548       0.499120      0.005704
9         0.581321       0.001034       0.500217      0.005156
10        0.583273       0.002048       0.500971      0.005279
11        0.584110       0.002398       0.498692      0.003015
12        0.585233       0.002750       0.499732      0.003468
13        0.585603       0.002882       0.500918      0.003058
14        0.588701       0.003346       0.499292      0

    train-auc-mean  train-auc-std  test-auc-mean  test-auc-std
0         0.519743       0.019014       0.498476      0.001133
1         0.548002       0.006959       0.493611      0.001581
2         0.565036       0.001534       0.496487      0.002232
3         0.568835       0.002173       0.496701      0.006005
4         0.572453       0.000948       0.498599      0.005499
5         0.575439       0.001543       0.496864      0.009514
6         0.576649       0.001167       0.498268      0.006990
7         0.577425       0.001815       0.498991      0.003886
8         0.579199       0.000548       0.499120      0.005704
9         0.581321       0.001034       0.500217      0.005156
10        0.583273       0.002048       0.500971      0.005279
11        0.584110       0.002398       0.498692      0.003015
12        0.585233       0.002750       0.499732      0.003468
13        0.585603       0.002882       0.500918      0.003058
14        0.588701       0.003346       0.499292      0

    train-auc-mean  train-auc-std  test-auc-mean  test-auc-std
0         0.542229       0.037562       0.494696      0.006921
1         0.683834       0.009370       0.495135      0.005362
2         0.749785       0.004616       0.494153      0.006982
3         0.805221       0.002983       0.492795      0.005438
4         0.841125       0.003742       0.489472      0.005027
5         0.866706       0.004243       0.487291      0.004402
6         0.885667       0.005220       0.487194      0.004583
7         0.898679       0.002672       0.491258      0.003759
8         0.910898       0.002271       0.494550      0.006147
9         0.920152       0.002095       0.494495      0.000524
10        0.927780       0.001562       0.494028      0.002055
11        0.935070       0.001700       0.494640      0.002920
12        0.941740       0.002072       0.494181      0.001885
13        0.948016       0.002328       0.495517      0.001071
14        0.952943       0.001784       0.499132      0

    train-auc-mean  train-auc-std  test-auc-mean  test-auc-std
0         0.519743       0.019014       0.498476      0.001133
1         0.548583       0.007780       0.492774      0.002098
2         0.564787       0.001474       0.495167      0.004307
3         0.568768       0.001966       0.497327      0.004841
4         0.574423       0.001305       0.495155      0.004106
5         0.575649       0.001353       0.495260      0.002723
6         0.575777       0.000336       0.496784      0.002723
7         0.579036       0.001457       0.497509      0.004461
8         0.579929       0.002731       0.496928      0.003695
9         0.581464       0.001734       0.497577      0.002873
10        0.583313       0.001926       0.495932      0.003294
11        0.584877       0.001312       0.498510      0.003638
12        0.586812       0.001516       0.500023      0.003113
13        0.587890       0.001766       0.498566      0.001975
14        0.588481       0.001662       0.498771      0

In [109]:
params = xgb_bo.max['params']

print(params)

#Conversting the max_depth and n_estimator values from float to int
params['max_depth']= int(round(params['max_depth']))
#params['gamma']= params['gamma']
params['n_estimators']= int(round(params['n_estimators']))
params['min_child_weight']= int(round(params['min_child_weight']))
#params['max_bin']= int(round(params['max_bin']))
print(params)

{'max_depth': 21.217250104407967, 'min_child_weight': 56.164669171924835, 'n_estimators': 272.3521687774187}
{'max_depth': 21, 'min_child_weight': 56, 'n_estimators': 272}


In [110]:
model_eval(xgb.XGBClassifier(**params),x_train,y_train,x_test,y_test)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=21,
              min_child_weight=56, missing=None, n_estimators=272, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)


 Accuracy score of train:  0.8708925449410498
 Accuracy score of test:  0.7946867482480912


 Confusion Matrix of train:  [[9473 1690]
 [1190 9954]]
 Confusion Matrix of test:  [[3612 1159]
 [ 804 3986]]


 Auc of train:  0.9408102687906201
 Auc of test:  0.8627039494440357


In [69]:
rfc = RandomForestClassifier()
model_eval(rfc,x_train,y_train,x_test,y_test)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


 Accuracy score of train:  0.9996109252198272
 Accuracy score of test:  0.866557734204793


 Confusion Matrix of train:  [[11154     0]
 [    5  1692]]
 Confusion Matrix of test:  [[4768   12]
 [ 723    5]]


 Auc of train:  0.9999993131990775
 Auc of test:  0.48863783967078944


In [113]:
params = {
    'n_estimators' : sp_randint(5,150),
    'max_depth': sp_randint(2,10),
    'max_features' : sp_randint(1,15),
    'min_samples_split' : sp_randint(2,50),
    'min_samples_leaf' : sp_randint(1,50),
    'criterion' : ['gini','entropy']}

rsearch_rf = RandomizedSearchCV(rfc,param_distributions=params,n_iter=100,n_jobs=-1,cv=3,scoring='roc_auc',random_state=1)
rsearch_rf.fit(x_train_os, y_train_os)

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [71]:
rsearch_rf.best_params_

{'criterion': 'entropy',
 'max_depth': 7,
 'max_features': 5,
 'min_samples_leaf': 28,
 'min_samples_split': 39,
 'n_estimators': 62}

In [72]:
rfc = RandomForestClassifier(**rsearch_rf.best_params_)
model_eval(rfc,x_train,y_train,x_test,y_test)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=7, max_features=5,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=28, min_samples_split=39,
                       min_weight_fraction_leaf=0.0, n_estimators=62,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


 Accuracy score of train:  0.867948019609369
 Accuracy score of test:  0.8678286129266521


 Confusion Matrix of train:  [[11154     0]
 [ 1697     0]]
 Confusion Matrix of test:  [[4780    0]
 [ 728    0]]


 Auc of train:  0.7116231229598711
 Auc of test:  0.4795992057106074


In [73]:
knn_train['target'].value_counts()

0    15934
1     2425
Name: target, dtype: int64

In [100]:
xy_train0 = knn_train.loc[knn_train['target']==0]
xy_train1 = knn_train.loc[knn_train['target']==1]

len0 = len(xy_train0)
len1 = len(xy_train1)

print('len0 ',len0 ,'len1 ',len1 )

len0  15934 len1  2425


In [88]:
x_train.shape

(12851, 12)

In [98]:
xy_train.shape

(16685, 13)

In [104]:
xy_train_os.columns

Index(['enrollee_id', 'city', 'city_development_index', 'gender',
       'relevent_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours', 'target'],
      dtype='object')

In [106]:
xy_train1_os = xy_train1.sample(len0,replace=True, random_state=1)
xy_train_os = pd.concat([xy_train1_os,xy_train0],axis=0)

x_train_os = xy_train_os.drop(['enrollee_id','target'],axis=1)
y_train_os = xy_train_os['target']
x_train, x_test, y_train, y_test = train_test_split(x_train_os, y_train_os, test_size=0.3, random_state=9)

In [102]:
print(xy_train_os['target'].value_counts())

1    15934
0    15934
Name: target, dtype: int64


In [112]:
rfc = RandomForestClassifier()
model_eval(rfc,x_train,y_train,x_test,y_test)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


 Accuracy score of train:  0.9995068812480388
 Accuracy score of test:  0.9595230624411673


 Confusion Matrix of train:  [[11154     9]
 [    2 11142]]
 Confusion Matrix of test:  [[4438  333]
 [  54 4736]]


 Auc of train:  0.9999913022838048
 Auc of test:  0.9931982283358619


In [114]:
rfc = RandomForestClassifier(**rsearch_rf.best_params_)
model_eval(rfc,x_train,y_train,x_test,y_test)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=9, max_features=6,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=6, min_samples_split=38,
                       min_weight_fraction_leaf=0.0, n_estimators=106,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


 Accuracy score of train:  0.7257363159546331
 Accuracy score of test:  0.6877941637903985


 Confusion Matrix of train:  [[8706 2457]
 [3661 7483]]
 Confusion Matrix of test:  [[3505 1266]
 [1719 3071]]


 Auc of train:  0.8134446065445796
 Auc of test:  0.7630763498502827


In [118]:
lr = LabelEncoder()
cs = [i for i in test.columns if test[i].dtype == 'object']
for c in cs:
    lr.fit(list(test[c].values))
    test[c] = lr.transform(list(test[c].values))

In [119]:
col = ['enrollee_id', 'city', 'city_development_index', 'gender',
       'relevent_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours']


In [120]:

knn_test = pd.DataFrame(np.round(imputor.fit_transform(test)),columns=col)


In [121]:

colu = list(knn_test.columns)
for i in colu:
    if (i!='city_development_index'):
        knn_test[i]=knn_test[i].astype(int)

In [ ]:
xy_train_os

In [131]:
test_new = knn_test.drop('enrollee_id',axis=1)

In [132]:
pred=rfc.predict(test_new)

In [133]:
sub = pd.read_csv('sample_submission_sxfcbdx.csv')
sub.head()

,enrollee_id,target
0,16548,0
1,12036,0
2,11061,0
3,5032,0
4,17599,0


In [135]:
sub.shape,test_new.shape

((15021, 2), (15021, 12))

In [137]:
sub["target"]=pred

In [138]:
sub[['enrollee_id', 'target']].to_csv('submission.csv',index=False)